# Import

In [1]:
import sys

%pip install joblib==1.2.0
%pip install  pycaret
%pip install scikit-learn

%pip install seaborn

%pip install pytorch-lightning

%pip install plotly

%pip install nltk

%pip install unidecode



Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:

import pandas as pd
import numpy as np
import statistics
import json
import torch
import pytorch_lightning as pl
import re, nltk
import torch
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px
import plotly.graph_objects as go

from pycaret.classification import  *

from statistics import mode
from unidecode import unidecode

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from torch import Tensor

from torch.nn import Module, Linear, Dropout, Sequential
from torch.nn.functional import cross_entropy, relu, sigmoid, binary_cross_entropy

from torch.optim import Adam, Optimizer
from torch.utils.data import Dataset, DataLoader, random_split

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


from typing import Tuple, List, Callable

from numpy import nan, percentile, asarray
from numpy.typing import ArrayLike
from numpy.random import randn


from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD



# Carga de Datos

In [3]:
df = pd.read_csv('Data_completa_enrriquecida.csv',  index_col=0)
df.head()

,Idioma,lyrics_state,hasLetra,Lyrics,Lyrics_proces,songId,track_uri,track_name,artist_uri,artist_info,...,dominance_tags,mbid,spotify_id,genre,cuadrante,cuadReal,lexicones,emociones,emocionesLetra,emocion_mas_comun
Letra,,,,,,,,,,,,,,,,,,,,,
Data/Letras/1-'Till-I-Collapse-Eminem.txt,en,complete,1.0,TranslationsEspañolTürkçePortuguêsItalianoहिन्...,cause feel tired left left feel weak feel weak...,till-i-collapse-eminem.txt,spotify:track:4xkOaSrkexMciUUogZKVTS,Till I Collapse,spotify:artist:7dGJo4pcD2V6oG8kP0tJRR,{'external_urls': {'spotify': 'https://open.sp...,...,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap,3,2,[0. 2. 0. 0. 0. 2. 6. 0. 0. 2.],"['anticipation', 'negative', 'positive', 'trust']","['3', '2', '1', '3']",3
Data/Letras/2-St.-Anger-Metallica.txt,en,complete,1.0,St. Anger Lyrics[Verse]\nSaint Anger 'round my...,saint anger round neck saint anger round neck ...,st.-anger-metallica.txt,spotify:track:3fOc9x06lKJBhz435mInlH,St. Anger,spotify:artist:2ye2Wgw4gimLv2eAKyk1NB,{'external_urls': {'spotify': 'https://open.sp...,...,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,3,2,[0. 6. 0. 0. 0. 6. 0. 0. 0. 0.],"['anticipation', 'negative']","['3', '2']",3
Data/Letras/3-Speedin'-Rick-Ross.txt,en,complete,1.0,Speedin’ Lyrics[Intro: Rick Ross]\nLegendary\n...,legendary runners know trilla dollar count acc...,speedin-rick-ross.txt,spotify:track:3Y96xd4Ce0J47dcalLrEC8,Speedin',spotify:artist:1sBkRIssrMs1AbVkOJbc7a,{'external_urls': {'spotify': 'https://open.sp...,...,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap,3,2,[0. 2. 0. 0. 0. 2. 0. 0. 0. 0.],"['anticipation', 'negative']","['3', '2']",3
Data/Letras/4-Bamboo-Banga-M.I.A..txt,en,complete,1.0,"Bamboo Banga Lyrics[Intro]\nRoad runner, road ...",road runner road runner going miles hour road ...,bamboo-banga-m.i.a..txt,spotify:track:6tqFC1DIOphJkCwrjVzPmg,Bamboo Banga,spotify:artist:0QJIPDAEDILuo8AIq3pMuU,{'external_urls': {'spotify': 'https://open.sp...,...,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop,3,1,[0. 0. 0. 0. 0. 0. 4. 0. 0. 0.],['positive'],['1'],1
Data/Letras/5-Die-MF-Die-Dope.txt,en,complete,1.0,Die MF Die Lyrics[Intro]\nDie!\n\n[Verse 1]\nI...,die need forgiveness need hate need acceptance...,die-mf-die-dope.txt,spotify:track:5bU4KX47KqtDKKaLM4QCzh,Die MF Die,spotify:artist:7fWgqc4HJi3pcHhK8hKg2p,{'external_urls': {'spotify': 'https://open.sp...,...,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal,3,1,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0.],"['anticipation', 'negative']","['3', '2']",3


## preprocesamiento

In [4]:
df = df[['Idioma', 'lyrics_state', 'hasLetra', 'Lyrics', 'Lyrics_proces',
       'songId', 'track_uri', 'emocion_mas_comun', 'cuadrante',
       'cuadReal']]

In [5]:
df.Idioma.value_counts()

en              23460
es                387
fr                239
pt                214
de                183
it                102
sv                 36
nl                 36
la                 34
pl                 28
da                 19
no                 15
romanization       10
ko                  9
tr                  6
id                  6
ja                  4
ca                  3
ro                  3
hr                  3
ar                  3
fi                  3
xh                  2
sq                  2
eu                  2
af                  2
sw                  2
ln                  2
gd                  2
ga                  2
ru                  2
ia                  1
el                  1
sco                 1
zu                  1
ms                  1
ss                  1
ak                  1
lv                  1
tl                  1
gl                  1
bs                  1
wo                  1
ve                  1
yo                  1
sm        

In [6]:
df = df[df["Idioma"]=='en']

In [7]:
corpus = df.Lyrics_proces

In [8]:
df.shape

(23460, 10)

# Extracción de Características

In [9]:
corpus_df = df[['Lyrics', 'cuadReal']]
corpus_df = corpus_df.drop_duplicates()
corpus_df = corpus_df.reset_index()
corpus_df = corpus_df[['Lyrics', 'cuadReal']]


In [10]:
s = setup(corpus_df, 
	target = 'cuadReal', 
	session_id = 123, 
	text_features_method= 'tf-idf', 
	text_features = ['Lyrics'], 
	ignore_features = ['Letra'],
	fix_imbalance = True
)

MemoryError: Unable to allocate 30.6 GiB for an array with shape (271453, 15135) and data type float64

## Tfidf

In [11]:
vectorizer = TfidfVectorizer(
    sublinear_tf=True, min_df=70, max_df=0.6
    )

In [12]:
tfidf_matrix = vectorizer.fit_transform(corpus)

In [13]:
joblib.dump(tfidf_matrix, "vect_tfidf.joblib")

['vect_tfidf.joblib']

In [ ]:
tfidf_matrix = joblib.load("vect_tfidf.joblib") 

## LSA

In [ ]:
# Aplicar LSA con TruncatedSVD
lsa_model = TruncatedSVD(n_components=5)  # Especifica el número de componentes deseados
lsa_matrix = lsa_model.fit_transform(tfidf_matrix)

In [ ]:
joblib.dump(lsa_matrix, "lsa_matrix.joblib")

In [ ]:
lsa_matrix = joblib.load("lsa_matrix.joblib") 

# Dividir datos

In [ ]:
class PersonalityDataset(Dataset):
    def __init__(self, features: ArrayLike, labels: ArrayLike):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.int64)

    def __len__(self) -> int:
        return self.features.shape[0]

    def __getitem__(self, idx: int) -> Tuple[Tensor, Tensor]:
        return self.features[idx], self.labels[idx]

In [ ]:
features = vect.transform(corpus).toarray()
labels = df.cuadReal.to_numpy()

In [ ]:
labels.shape

In [ ]:
features.shape

In [ ]:
ds = PersonalityDataset(
        features = features, labels=labels
        ) 
train_ds, val_ds, test_ds = random_split(ds, [0.6, 0.2, 0.2])

In [ ]:
train_dl = DataLoader(train_ds, batch_size=1024) 
val_dl = DataLoader(val_ds, batch_size=512) 
test_dl = DataLoader(test_ds, batch_size=512) 

In [ ]:
(train_ds.dataset.features).shape, train_ds.dataset.labels.shape

# Modelamiento

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=15, random_state=0)

model.fit(train_ds.dataset.features, train_ds.dataset.labels)
predicted = model.predict(train_ds.dataset.features)

print(f'El score del modelo  es: {model.score(train_ds.dataset.features, train_ds.dataset.labels)}')

## árboles de decisión

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

model = DecisionTreeClassifier(max_depth=5)

clf = model.fit(train_ds.dataset.features, train_ds.dataset.labels) 
predicted = model.predict(train_ds.dataset.features)

print(f'El score del modelo  es: {model.score(train_ds.dataset.features, train_ds.dataset.labels)}')

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=10)

model.fit(train_ds.dataset.features, train_ds.dataset.labels)
predicted = model.predict(train_ds.dataset.features)

print(f'El score del modelo  es: {model.score(train_ds.dataset.features, train_ds.dataset.labels)}')

## regresión logística

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', max_iter=10)

model.fit(train_ds.dataset.features, train_ds.dataset.labels)
predicted = model.predict(train_ds.dataset.features)

print(f'El score del modelo es: {model.score(train_ds.dataset.features, train_ds.dataset.labels)}')

balancear -> smote (data aumentation)
revisar las proporciones de los cuadrantes
usar modelo preentrenado
probar random forest

In [ ]:
Batch = Tuple[Tensor, Tensor]

class Dense(Module):
    def __init__(
        self,
        in_features: int,
        out_features: int,
        activation: Callable,
        dropout: float,
        ):
        super(Dense, self).__init__()
        self.layer = Linear(in_features=in_features, out_features=out_features)
        self.dropout = Dropout(p=dropout)
        self.activation = activation
    
    def forward(self, x: Tensor) -> Tensor:
        return self.activation(self.dropout(self.layer(x)))

class MLP(LightningModule):
    def __init__(
            self, n_units: List[int], activations: List[Callable],
            dropout: float
            ):
        super(MLP, self).__init__()
        self.layers = Sequential(
            *(
                Dense(
                    in_features=ins,
                    out_features=outs,
                    activation=act,
                    dropout=dropout
                    )
                for ins, outs, act in zip(
                    n_units,
                    n_units[1:],
                    activations
                )
            )
        )
    
    def forward(self, x: Tensor) -> Tensor:
        return self.layers(x)

    def configure_optimizers(self) -> Optimizer:
        return Adam(self.parameters(), lr=1e-4)

    def training_step(
        self, train_batch: Batch, batch_idx: int
        ) -> Tensor:
        features, labels = train_batch
        y_pred = self.forward(features)
        loss_val = cross_entropy(y_pred, labels)
        self.log("train_loss", loss_val)
        return loss_val
    
    def validation_step(
        self, val_batch: Batch, batch_idx: int
        ):
        features, labels = val_batch
        y_pred = self.forward(features)
        loss_val = cross_entropy(y_pred, labels)
        self.log("val_loss", loss_val)

In [ ]:
logger = CSVLogger(save_dir="logs/", name="losses.csv")
trainer = Trainer(
    max_epochs=300,
    logger=logger, log_every_n_steps=10,
    callbacks=[
        EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=20),
        ]
)

In [ ]:
linear = lambda x: x
model = MLP(
    n_units=[features.shape[1], 16],
    activations=[linear],
    dropout=0.0
    )

In [ ]:
trainer.fit(model, train_dl, val_dl)

# Evaluación

In [ ]:
df = pd.read_csv("logs/losses.csv/version_0/metrics.csv")
df_train = (
    df
    .filter(["epoch", "train_loss"])
    .dropna()
    .groupby("epoch")
    .agg({"train_loss": "mean"})
)
df_test = (
    df
    .filter(["epoch", "val_loss"])
    .dropna()
    .groupby("epoch")
    .agg({"val_loss": "mean"})
)   

df = df_train.join(df_test, on="epoch").reset_index()

In [ ]:
fig = (
    px.line(df, x="epoch", y=["train_loss", "val_loss"])
)
fig.show()

In [ ]:
metric = []
for features_batch, labels_batch in test_dl:
    y_pred = torch.argmax(model(features_batch), dim=1)
    metric.append(y_pred == labels_batch)

In [ ]:
metric = torch.concat(metric) 
print(metric.float().mean())